# Notebook E-tivity 1 CE4021

Student name: Sean Meade

Student ID: 10128921

<hr style="border:2px solid gray"> </hr>

## Imports

In [ ]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

You may add as many cells as you require to complete this task. Refer to the Rubric for E-tivity 1 to see how you will be assessed.

Showing excellent understanding of polynomial derivatives / integrals.

### Understanding Polynomial derivatives

#### What are polynomials?

A polynomial is a mathematical equation consisting of variables and coefficients. A polynomial can have any number of coefficients and variables, and the variables can have any order of power that is a positive integer.

Example of a polynomial: $ y(x) = 3x^3 - 5x^2 + 2x - 1 $

##### Equation for Vertical Motion

![Vertical height vs. time taken](https://i.stack.imgur.com/nrp41.png)

Polynomials are equations that can be used to describe linear processes is one that represents a ball being fired directly up into the air, also known as the "equation for vertical motion". For the sake of this example we will fire our ball within a vacuum (i.e. it is not effected by air currents, wind resistance etc). The following equation is a polynomial that represents the vertical displacement ($ h $) as a function of time ($ t $) with respect to the initial velocity ($ V_{iy} $), and acceleration/deceleration due to gravity ($ g $).

\begin{equation}
    h(t) = V_{iy}t - \frac{1}{2}gt^2
\end{equation}

#### Differentiation of a polynomial

What if I wanted to find the velocity of the ball ($ V_{y} $) at a time ($ t $)? Well the velocity can be calculated as the change in displacement ($ ds $) divided by the change in time ($ dt $) or $ \frac{ds}{dt} $ 

$$ \frac{change~in~displacement}{change~in~time} = \frac{ds}{dt} = velocity $$

The derivative is defined as 'an instantaneous rate of change' which, as pointed out in the [video by 3Blue1Brown on the topic](https://youtu.be/9vKqVkMQHKk?si=QGEQitIiNg3zORRg&t=30s), is an oxymoron. "Change is something that happens between separate points in time, and when you blind yourself to all but a single instant, there is no more room for change." For example if you wanted to get the velocity at 3 minutes (at an instant in time) and the velocity requires a change in time to calculate it then it really doesn't make sense at face value.

$ dt $ is an increment of time that approaches zero ($ {dt \to 0} $). To find the velocity at dt we get the distance at time $ t + dt $, subtract the distance at $ t $, and divide by the time $ dt $. 

TODO: Add image showing dt change

\begin{equation}
    \frac{dh}{dt} = \frac{h(t + dt) - h(t)}{dt}
    \tag{2}
\end{equation}

\begin{equation}
    \frac{dh}{dt} = \frac{(V_{iy}(t + dt) - \frac{1}{2}g(t + dt)^2) - (V_{iy}t - \frac{1}{2}gt^2)}{dt}
    \tag{3}
\end{equation}

\begin{equation}
    \frac{dh}{dt} = \frac{V_{iy}(t) + V_{iy}(dt) - \frac{1}{2}gt^2 - \frac{1}{2}g(dt)^2 - g(t)(dt) - V_{iy}(t) + \frac{1}{2}gt^2}{dt}
    \tag{4}
\end{equation}

\begin{equation}
    \frac{dh}{dt} = \frac{(V_{iy}(dt) - \frac{1}{2}g(dt)^2 - g(t)(dt)}{dt}
    \tag{5}
\end{equation}

\begin{equation}
    \frac{dh}{dt} = V_{iy} - \frac{1}{2}g(dt) - g(t)
    \tag{6}
\end{equation}


As $ {dt \to 0} $ the middle term approaches zero.


\begin{equation}
    \frac{dh}{dt} = V_{iy} - g(t)
    \tag{7}
\end{equation}

With the above equation we can work out the 'instantaneous velocity'.

There is a rule for this type of differentiation named the power rule. Taking the positive integer power of the variable, multiplying that by the whole term and then taking 1 from that power:

\begin{equation}
    \frac{d}{dx}(ax^n) = anx^{n-1}
    \tag{The Power Rule}
\end{equation}

You will note that if applied to equation 1 it can produce equation 7 with one or two easy steps. Applying the power rule to differentiate each term of the equation for vertical motion:

$$ \frac{dh}{dt} = (1)V_{iy}t^{1-1} - (2)\frac{1}{2}gt^{2-1} $$

And simplifying we get equation 7 that calculates velocity:

$$ \frac{dh}{dt} = V_{iy} - g(t) $$

This rule is what is used to calculate the derivatives in the *polynomial_differentiation* method below.

(TODO: Graph showing distance and speed together and how they are related)





### Implementation

Correctly working implementation with efficient use of flow control elements, data types and operators. Clean code with appropriate variable names.  Appropriate documentation of each function.

In [2]:
# Maybe spread each step out so process is more clear
x = [[1, 2], [3, 4], [7, 8]]

def polynomial_differentiation(input_list):

    output_polynomial = []

    for coefficient, var_power in input_list:
        output_polynomial.append([coefficient * var_power, var_power - 1])

    return(output_polynomial)

def evaluate_polynomial(input_list, known_var):

    output_poly = polynomial_differentiation(input_list)

    sum = 0
    if known_var:
        for coefficient, power in output_poly:
            sum += (known_var**power)*coefficient

    return sum
        

print(polynomial_differentiation(x))
print(evaluate_polynomial(x, 5))

[[2, 1], [12, 3], [56, 7]]
4376510


### Testing

Test cases cover corner cases without redundancy.

<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.